# Homework Assignment 2: Network Transshipment Problem

**Reed Ballesteros**

**MSDS-460**

**4/24/2022**

**Instructor: Prof. Thomas Miller**

A UK brewing company owns four breweries and three packaging facilities and ships its products to fifteen demand locations (retail stores and pubs). Beer product flows from breweries to packaging facilities and then from packaging facilities to demand locations. Assume that no beer product goes directly from breweries to demand locations. Costs vary between pairs of locations.   

Mathematical programming can be used for supply chain optimization. For the brewery case, we can solve for a minimum-cost plan for shipping across breweries, packaging facilities, and demand points. We aggregate data across products to show the minimum and maximum quantities of beer that can be produced at each brewery, minimum and maximum quantities that can be processed at packaging facilities, and quantities on order at demand points. We simplify the problem by summing quantities and averaging costs across beer types (ale and lager) and container/packaging types.

Complete data for the complete brewery case are provided in Kallrath (2021), which is available on Course Reserves. Note that the case presented here is a simplification of the complete brewery case, as we are considering total liquid being shipped from one location to another. We make no distinction between ale and lager. We make no distinctions across packaging types. Regardless, the aggregate liquid values used in this assignment are consistent with data for the complete brewery case. 

Table 1 shows transportation costs between breweries and packaging facilities. Table 2 shows transportation costs between packaging facilities and demand locations. Tables 3a and 3b show brewery and packaging facility capacities. Table 4 shows the units on order at each demand point. 

Table 1. Transportation Costs between Breweries to Packaging Facilities

Brewery|Packaging Facility|Transportation Cost<br> per Unit
---|---|---
1|	1|	1.55
1|	2|	0.51
1|	3|	0.9
2|	1|	0.81
2|	2|	3.18
2|	3|	0.65
3|	1|	2.13
3|	2|	0.97
3|	3|	0.51
4|	1|	1.23
4|	2|	2.15
4|	3|	2.08

Table 2. Transportation Costs from Packaging Facilities to Demand Points

Packaging Facility|Demand Point|Average Transportation<br> Cost per Unit
---|---|---
4.82|	1.83|	2.66
2.05|	4.03|	0.95
4.42|	3.95|	3.94
3.83|	4.21|	2.04
0.97|	4.78|	2.35
3.04|	3.2|	1.42
3.91|	1.88|	3.6
4.03|	2.96|	3.17
5.11|	5.11|	1.34
0.9|	2.67|	4.51
4.39|	4.14|	0.74
0.85|	1.22|	0.94
2.81|	5.1|	1.98
3.94|	3.47|	4.77
1.04|	1.92|	2.04


Table 3a Brewery and Packaging Facility Capacities

Brewery|	Minimum Units|	Maximum Units
---|---|---
1|	100|	2000
2|	150|	2500
3|	200|	3500
4|	100|	2000

Table 3b Brewery and Packaging Facility Capacities

Packaging Facility|	Minimum Units|	Maximum Units
---|---|---
1|	50|	500
2|	100|	1500
3|	150|	2500

Table 4. Demand Point Units Ordered

Demand Point|Units Ordered
---|---
1|48
2|84
3|64
4|106
5|47
6|57
7|64
8|93
9|74
10|41
11|61
12|42
13|57
14|70
15|41

Total transportation/shipping costs depend on the number of units shipped across each of 57 paths between locations: 12 paths between breweries and packaging facilities, and 45 paths between packaging facilities and demand points. The objective of supply chain optimization is to minimize total transportation costs.  

To solve this supply chain optimization problem, let's consider using a Python program that draws on the PuLP package for mathematical programming. 

As part of our program, let's include a multiplier for demand, so we can see how demand affects the solution.

Output from the supply chain optimization should include optimal numbers of units of beer product to be shipped across each of the 57 paths between locations. We should also review counts for total units of production from each of the four brewery locations and each of the three packaging facilities. 

To complete this assignment, review the program output and answer these five questions:  

(1) Solve the supply chain optimization problem with initial settings of parameters for brewing, packaging, and demand. Describe the solution by providing the total cost (minimum cost) and quantities of beer being shipped between each pair of locations. 

In [97]:
import regex as re # regular expresstions used in manipulating output for reporting solution
import pulp # mathematical programming

In [98]:
# Define matices and hard-coded constants here

demand_multiplier = 1  # default is 1

brewery = ["B1", "B2", "B3", "B4"]

# Dictionary for maximumn units/capacity in tons
brewery_minimum = {"B1": 100,
           "B2": 150,
           "B3": 200,
           "B4": 100}

brewery_maximum = {"B1": 2000,
           "B2": 2500,
           "B3": 3500,
           "B4": 2000}

packaging_facility = ["PF1", "PF2", "PF3"]

packaging_facility_minimum = {"PF1":50,
          "PF2":100,
          "PF3":150}

packaging_facility_maximum = {"PF1":600,
          "PF2":1500,
          "PF3":2500}

demand_point = ["DP1", "DP2", "DP3", 
                "DP4", "DP5", "DP6", 
                "DP7", "DP8", "DP9", 
                "DP10", "DP11", "DP12", 
                "DP13", "DP14", "DP15"]

demand_point_units_orderd = {"DP1": 48, "DP2": 84, "DP3": 64, 
               "DP4": 106, "DP5": 47, "DP6": 57,
               "DP7": 64, "DP8": 93, "DP9": 74,
               "DP10": 41, "DP11": 61, "DP12": 42,
               "DP13": 57, "DP14": 70, "DP15": 41}

brewery_to_packaging_facility_shipping_costs = [
         [1.55, 0.51, 0.9],  # "B1" Packaging Facilities in rows
         [0.81, 3.18, 0.65], # "B2" Packaging Facilities in rows
         [2.13, 0.97, 0.51], # "B3" Packaging Facilities in rows
         [1.23, 2.15, 2.08]] # "B4" Packaging Facilities in rows


packaging_facility_to_demand_point_shipping_costs = [
        # "PF1" Demand Point in rows
        [4.82, 2.05, 4.42, 3.83, 0.97, 3.04, 3.91, 4.03, 5.11, 0.9, 4.39, 0.85, 2.81, 3.94, 1.04],
        # "PF2" Demand Point in rows
        [1.83, 4.03, 3.95, 4.21, 4.78, 3.2, 1.88, 2.96, 5.11, 2.67, 4.14, 1.22, 5.1, 3.47, 1.92],
        # "PF3" Demand Point in rows
        [2.66, 0.95, 3.94, 2.04, 2.35, 1.42, 3.6, 3.17, 1.34, 4.51, 0.74, 0.94, 1.98, 4.77, 2.04]]

demand = demand_point_units_orderd
for key in list(demand_point_units_orderd.keys()):
    demand[key] = demand_multiplier * demand_point_units_orderd[key]

DP1_demand = demand['DP1'] 
DP2_demand = demand['DP2'] 
DP3_demand = demand['DP3'] 
DP4_demand = demand['DP4'] 
DP5_demand = demand['DP5'] 
DP6_demand = demand['DP6'] 
DP7_demand = demand['DP7'] 
DP8_demand = demand['DP8'] 
DP9_demand = demand['DP9'] 
DP10_demand = demand['DP10'] 
DP11_demand = demand['DP11'] 
DP12_demand = demand['DP12'] 
DP13_demand = demand['DP13'] 
DP14_demand = demand['DP14'] 
DP15_demand = demand['DP15'] 

total_demand = sum(demand.values())

In [109]:
# Pulp calculations here

prob = pulp.LpProblem("Distribution_1", pulp.LpMinimize)
solver = pulp.getSolver("PULP_CBC_CMD") # available on Windows computers

first_costs = pulp.makeDict([brewery,packaging_facility],brewery_to_packaging_facility_shipping_costs,0)
second_costs = pulp.makeDict([packaging_facility,demand_point],packaging_facility_to_demand_point_shipping_costs,0)

# Create the 'prob' variable to contain the problem data
prob = pulp.LpProblem("Distribution_1",pulp.LpMinimize)

# Create list of tuples containing all the possible brewery-to-packaging facility routes for transport
first_routes = [(i,j) for i in brewery for j in packaging_facility]
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
first_vars = pulp.LpVariable.dicts("route",(brewery,packaging_facility),0,None,pulp.LpInteger)

# Create list of tuples containing all the possible packaging facility-to-demand point for transport
second_routes = [(i,k) for i in packaging_facility for k in demand_point]
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
second_vars = pulp.LpVariable.dicts("route",(packaging_facility,demand_point),0,None,pulp.LpInteger)

for i in brewery:
    prob += pulp.lpSum([first_vars[i][j] for j in packaging_facility]) <= brewery_maximum[i], "Brewery_Capacity%s"%i

for i in brewery:
    prob += pulp.lpSum([first_vars[i][j] for j in packaging_facility]) >= brewery_minimum[i], "Brewery_Minimum%s"%i

for j in packaging_facility:
    prob += pulp.lpSum([first_vars[i][j] for i in brewery]) <= packaging_facility_maximum[j], "Packaging_Facility_Capacity%s"%j

for j in packaging_facility:
    prob += pulp.lpSum([first_vars[i][j] for i in brewery]) >= packaging_facility_minimum[j], "Packaging_Facility_Minimum%s"%j
        
for k in demand_point:
    prob += pulp.lpSum([second_vars[j][k] for j in packaging_facility]) >= demand[k], "Meet_or_exceed_demand_point_input%s"%k


'''
# brewery-to-packaging facility routes
prob += pulp.LpVariable("route_B1_PF1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B1_PF2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B1_PF3", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B2_PF1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B2_PF2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B2_PF3", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B3_PF1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B3_PF2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B3_PF3", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B4_PF1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B4_PF2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_B4_PF3", lowBound=0, upBound=0, cat = 'Integer')

# packaging facility-to-demand point routes
prob += pulp.LpVariable("route_PF1_DP1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP3", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP6", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP5", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP6", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP7", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP8", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP9", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP10", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP11", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP12", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP13", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP14", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF1_DP15", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP3", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP6", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP5", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP6", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP7", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP8", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP9", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP10", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP11", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP12", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP13", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP14", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF2_DP15", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP1", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP2", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP3", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP6", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP5", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP6", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP7", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP8", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP9", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP10", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP11", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP12", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP13", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP14", lowBound=0, upBound=0, cat = 'Integer')
prob += pulp.LpVariable("route_PF3_DP15", lowBound=0, upBound=0, cat = 'Integer')
'''

# iteratively create route variables compared to by hand above
for (b,pf) in first_routes:
        prob += first_vars[b][pf]

for (pf,dp) in second_routes:
        prob += second_vars[pf][dp]

# The objective function for all transportation costs
prob += pulp.lpSum([first_vars[i][j]*first_costs[i][j] for (i,j) in first_routes]) + \
        pulp.lpSum([second_vars[i][k]*second_costs[i][k] for (i,k) in second_routes]), "All_Tansportation_Costs"

prob.solve()

# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimal value
for v in prob.variables():
    print(v.name, "=", round(v.varValue))
    

# print(prob.objective)


Status: Optimal
route_B1_PF1 = 0
route_B1_PF2 = 100
route_B1_PF3 = 0
route_B2_PF1 = 0
route_B2_PF2 = 0
route_B2_PF3 = 150
route_B3_PF1 = 0
route_B3_PF2 = 0
route_B3_PF3 = 200
route_B4_PF1 = 100
route_B4_PF2 = 0
route_B4_PF3 = 0
route_PF1_DP1 = 0
route_PF1_DP10 = 41
route_PF1_DP11 = 0
route_PF1_DP12 = 42
route_PF1_DP13 = 0
route_PF1_DP14 = 0
route_PF1_DP15 = 41
route_PF1_DP2 = 0
route_PF1_DP3 = 0
route_PF1_DP4 = 0
route_PF1_DP5 = 47
route_PF1_DP6 = 0
route_PF1_DP7 = 0
route_PF1_DP8 = 0
route_PF1_DP9 = 0
route_PF2_DP1 = 48
route_PF2_DP10 = 0
route_PF2_DP11 = 0
route_PF2_DP12 = 0
route_PF2_DP13 = 0
route_PF2_DP14 = 70
route_PF2_DP15 = 0
route_PF2_DP2 = 0
route_PF2_DP3 = 0
route_PF2_DP4 = 0
route_PF2_DP5 = 0
route_PF2_DP6 = 0
route_PF2_DP7 = 64
route_PF2_DP8 = 93
route_PF2_DP9 = 0
route_PF3_DP1 = 0
route_PF3_DP10 = 0
route_PF3_DP11 = 61
route_PF3_DP12 = 0
route_PF3_DP13 = 57
route_PF3_DP14 = 0
route_PF3_DP15 = 0
route_PF3_DP2 = 84
route_PF3_DP3 = 64
route_PF3_DP4 = 106
route_PF3_DP5 = 0
ro

In [108]:
prob

Distribution_1:
MINIMIZE
1.55*route_B1_PF1 + 0.51*route_B1_PF2 + 0.9*route_B1_PF3 + 0.81*route_B2_PF1 + 3.18*route_B2_PF2 + 0.65*route_B2_PF3 + 2.13*route_B3_PF1 + 0.97*route_B3_PF2 + 0.51*route_B3_PF3 + 1.23*route_B4_PF1 + 2.15*route_B4_PF2 + 2.08*route_B4_PF3 + 4.82*route_PF1_DP1 + 0.9*route_PF1_DP10 + 4.39*route_PF1_DP11 + 0.85*route_PF1_DP12 + 2.81*route_PF1_DP13 + 3.94*route_PF1_DP14 + 1.04*route_PF1_DP15 + 2.05*route_PF1_DP2 + 4.42*route_PF1_DP3 + 3.83*route_PF1_DP4 + 0.97*route_PF1_DP5 + 3.04*route_PF1_DP6 + 3.91*route_PF1_DP7 + 4.03*route_PF1_DP8 + 5.11*route_PF1_DP9 + 1.83*route_PF2_DP1 + 2.67*route_PF2_DP10 + 4.14*route_PF2_DP11 + 1.22*route_PF2_DP12 + 5.1*route_PF2_DP13 + 3.47*route_PF2_DP14 + 1.92*route_PF2_DP15 + 4.03*route_PF2_DP2 + 3.95*route_PF2_DP3 + 4.21*route_PF2_DP4 + 4.78*route_PF2_DP5 + 3.2*route_PF2_DP6 + 1.88*route_PF2_DP7 + 2.96*route_PF2_DP8 + 5.11*route_PF2_DP9 + 2.66*route_PF3_DP1 + 4.51*route_PF3_DP10 + 0.74*route_PF3_DP11 + 0.94*route_PF3_DP12 + 1.98*route

In [79]:
first_routes

[('B1', 'PF1'),
 ('B1', 'PF2'),
 ('B1', 'PF3'),
 ('B2', 'PF1'),
 ('B2', 'PF2'),
 ('B2', 'PF3'),
 ('B3', 'PF1'),
 ('B3', 'PF2'),
 ('B3', 'PF3'),
 ('B4', 'PF1'),
 ('B4', 'PF2'),
 ('B4', 'PF3')]

(2) Due to low demand for its products, the brewing company is thinking about closing any brewing location that is operating at minimum capacity. Reviewing the solution to the optimization problem, which brewery would you close (if any)?  

(3) Due to low demand, the company may also want to close one of its packaging facilities. Which packaging facility would you close (if any)? 

(4) Try multiplying demand by 2, 3, 4, or higher multiples. You can do this by modifying one line of the Python program. You can see the initial setting: demand_multiplier = 1  Setting the multiplier to 2, 3, 4, or higher values, will increase the level of demand, which will change the optimal solution. At what point does demand exceed the company's production capacity? At this point (full capacity), would you close any of the breweries or packaging facilities? 

(5) What have you learned from this supply chain optimization problem? Explain how you might apply methods of constrained optimization in your line of work.

The Distribution Planning for a Brewery case represents a constrained optimization problem (more specifically, a transshipment problem in mathematical programming). We are minimizing costs subject to production and demand constraints.  

Another way to approach a transshipment problem would be to consider the network structure across nodes for the breweries, packaging facilities, and demand points. We could vary shipping distances, times, or costs across paths (edges/links) between locations. A discrete event simulation could be used to trace the flow of beer products from node to node. A common discrete event simulation would model the transshipment problem as a network of queues. In week 7, we introduce discrete event simulation.

Are supply chain optimization and logistics management important to companies? You bet they are. Take note of this story from siliconANGLE:

https://siliconangle.com/2022/02/07/flexport-raises-935m-8b-valuation-logistics-management-platform/